In [1]:
import numpy as np
import matplotlib.pyplot as plt

import hdf5storage
import librosa

from model import *
from tb_utils import *
from preprocessing import *

from scipy import stats, signal
from sklearn.metrics import mean_squared_error

default_envelope_config = {
    'homomorphic_envelogram_with_hilbert': {'lpf_frequency': 8},
    'psd': {'fl_low': 40, 'fl_high': 60, 'resample': True},
    'wavelet': {'wavelet': 'db1',
                'levels': [4],
                'start_level': 1,
                'end_level': 5,
                'erase_pad': True}
}

In [2]:
#Load recordings
recording_2530, frequency = librosa.load('./2530_AV.wav', sr=None)
recording_14241, frequency = librosa.load('./14241_PV.wav', sr=None)
recording_23625, frequency = librosa.load('./23625_MV.wav', sr=None)
recording_24160, frequency = librosa.load('./24160_MV.wav', sr=None)
recording_40840, frequency = librosa.load('./40840_TV.wav', sr=None)

#Renna pre-processing
pre_proc_data_2530_orig=renna_preprocess_wave(input_signal=recording_2530, fs=4000, config_dict=default_envelope_config) #WITHOUT final standardization
pre_proc_data_14241_orig=renna_preprocess_wave(input_signal=recording_14241, fs=4000, config_dict=default_envelope_config)
pre_proc_data_23625_orig=renna_preprocess_wave(input_signal=recording_23625, fs=4000, config_dict=default_envelope_config)
pre_proc_data_24160_orig=renna_preprocess_wave(input_signal=recording_24160, fs=4000, config_dict=default_envelope_config)
pre_proc_data_40840_orig=renna_preprocess_wave(input_signal=recording_40840, fs=4000, config_dict=default_envelope_config)

In [3]:
#2530 Q32
envelopes_2530Q32 = [] #appending order is the same as the one in "preprocessing.py"
envelopes_2530Q32.append(np.squeeze(hdf5storage.loadmat('./2530_data/homo_test_2530.mat')['homo_test'])[0:pre_proc_data_2530_orig.shape[1]])
envelopes_2530Q32.append(np.squeeze(hdf5storage.loadmat('./2530_data/hilb_test_2530.mat')['hilb_test'])[0+1:pre_proc_data_2530_orig.shape[1]+1])
envelopes_2530Q32.append(np.squeeze(hdf5storage.loadmat('./2530_data/psd_test_2530.mat')['psd_test'])[0+1:pre_proc_data_2530_orig.shape[1]+1]) #I have to correct the amount of group delay in the logic, thus I do it here temporaney, visually assessed
envelopes_2530Q32.append(np.squeeze(hdf5storage.loadmat('./2530_data/swt_test_2530.mat')['swt_test'])[0:pre_proc_data_2530_orig.shape[1]])
envelopes_2530Q32 = np.stack(envelopes_2530Q32, axis=0)

#2530 Q16
envelopes_2530Q16 = []
envelopes_2530Q16.append(np.squeeze(hdf5storage.loadmat('./2530_data_Q16/homo_env_2530Q16.mat')['homo_env'])[0:pre_proc_data_2530_orig.shape[1]])
envelopes_2530Q16.append(np.squeeze(hdf5storage.loadmat('./2530_data_Q16/hilb_env_2530Q16.mat')['hilb_env'])[0+1:pre_proc_data_2530_orig.shape[1]+1])
envelopes_2530Q16.append(np.squeeze(hdf5storage.loadmat('./2530_data_Q16/psd_env_2530Q16.mat')['psd_env'])[0+1:pre_proc_data_2530_orig.shape[1]+1])
envelopes_2530Q16.append(np.squeeze(hdf5storage.loadmat('./2530_data_Q16/swt_env_2530Q16.mat')['swt_env'])[0:pre_proc_data_2530_orig.shape[1]])
envelopes_2530Q16 = np.stack(envelopes_2530Q16, axis=0)

#14241 Q32
envelopes_14241Q32 = []
envelopes_14241Q32.append(np.squeeze(hdf5storage.loadmat('./14241_data/homo_env_14241.mat')['homo_env'])[0:pre_proc_data_14241_orig.shape[1]])
envelopes_14241Q32.append(np.squeeze(hdf5storage.loadmat('./14241_data/hilb_env_14241.mat')['hilb_env'])[0+1:pre_proc_data_14241_orig.shape[1]+1])
envelopes_14241Q32.append(np.squeeze(hdf5storage.loadmat('./14241_data/psd_env_14241.mat')['psd_env'])[0+1:pre_proc_data_14241_orig.shape[1]+1])
envelopes_14241Q32.append(np.squeeze(hdf5storage.loadmat('./14241_data/swt_env_14241.mat')['swt_env'])[0:pre_proc_data_14241_orig.shape[1]])
envelopes_14241Q32 = np.stack(envelopes_14241Q32, axis=0)

#14241 Q16
envelopes_14241Q16 = []
envelopes_14241Q16.append(np.squeeze(hdf5storage.loadmat('./14241_data_Q16/homo_env_14241Q16.mat')['homo_env'])[0:pre_proc_data_14241_orig.shape[1]])
envelopes_14241Q16.append(np.squeeze(hdf5storage.loadmat('./14241_data_Q16/hilb_env_14241Q16.mat')['hilb_env'])[0+1:pre_proc_data_14241_orig.shape[1]+1])
envelopes_14241Q16.append(np.squeeze(hdf5storage.loadmat('./14241_data_Q16/psd_env_14241Q16.mat')['psd_env'])[0+1:pre_proc_data_14241_orig.shape[1]+1])
envelopes_14241Q16.append(np.squeeze(hdf5storage.loadmat('./14241_data_Q16/swt_env_14241Q16.mat')['swt_env'])[0:pre_proc_data_14241_orig.shape[1]])
envelopes_14241Q16 = np.stack(envelopes_14241Q16, axis=0)

#23625 Q32
envelopes_23625Q32 = []
envelopes_23625Q32.append(np.squeeze(hdf5storage.loadmat('./23625_data/homo_env_23625Q32.mat')['homo_env'])[0:pre_proc_data_23625_orig.shape[1]])
envelopes_23625Q32.append(np.squeeze(hdf5storage.loadmat('./23625_data/hilb_env_23625Q32.mat')['hilb_env'])[0+1:pre_proc_data_23625_orig.shape[1]+1])
envelopes_23625Q32.append(np.squeeze(hdf5storage.loadmat('./23625_data/psd_env_23625Q32.mat')['psd_env'])[0+1:pre_proc_data_23625_orig.shape[1]+1])
envelopes_23625Q32.append(np.squeeze(hdf5storage.loadmat('./23625_data/swt_env_23625Q32.mat')['swt_env'])[0:pre_proc_data_23625_orig.shape[1]])
envelopes_23625Q32 = np.stack(envelopes_23625Q32, axis=0)

#23625 Q16
envelopes_23625Q16 = []
envelopes_23625Q16.append(np.squeeze(hdf5storage.loadmat('./23625_data_Q16/homo_env_23625Q16.mat')['homo_env'])[0:pre_proc_data_23625_orig.shape[1]])
envelopes_23625Q16.append(np.squeeze(hdf5storage.loadmat('./23625_data_Q16/hilb_env_23625Q16.mat')['hilb_env'])[0+1:pre_proc_data_23625_orig.shape[1]+1])
envelopes_23625Q16.append(np.squeeze(hdf5storage.loadmat('./23625_data_Q16/psd_env_23625Q16.mat')['psd_env'])[0+1:pre_proc_data_23625_orig.shape[1]+1])
envelopes_23625Q16.append(np.squeeze(hdf5storage.loadmat('./23625_data_Q16/swt_env_23625Q16.mat')['swt_env'])[0:pre_proc_data_23625_orig.shape[1]])
envelopes_23625Q16 = np.stack(envelopes_23625Q16, axis=0)

#24160 Q32
envelopes_24160Q32 = []
envelopes_24160Q32.append(np.squeeze(hdf5storage.loadmat('./24160_data/homo_env_24160.mat')['homo_env'])[0:pre_proc_data_24160_orig.shape[1]])
envelopes_24160Q32.append(np.squeeze(hdf5storage.loadmat('./24160_data/hilb_env_24160.mat')['hilb_env'])[0+1:pre_proc_data_24160_orig.shape[1]+1])
envelopes_24160Q32.append(np.squeeze(hdf5storage.loadmat('./24160_data/psd_env_24160.mat')['psd_env'])[0+1:pre_proc_data_24160_orig.shape[1]+1])
envelopes_24160Q32.append(np.squeeze(hdf5storage.loadmat('./24160_data/swt_env_24160.mat')['swt_env'])[0:pre_proc_data_24160_orig.shape[1]])
envelopes_24160Q32 = np.stack(envelopes_24160Q32, axis=0)

#24160 Q16
envelopes_24160Q16 = []
envelopes_24160Q16.append(np.squeeze(hdf5storage.loadmat('./24160_data_Q16/homo_env_24160Q16.mat')['homo_env'])[0:pre_proc_data_24160_orig.shape[1]])
envelopes_24160Q16.append(np.squeeze(hdf5storage.loadmat('./24160_data_Q16/hilb_env_24160Q16.mat')['hilb_env'])[0+1:pre_proc_data_24160_orig.shape[1]+1])
envelopes_24160Q16.append(np.squeeze(hdf5storage.loadmat('./24160_data_Q16/psd_env_24160Q16.mat')['psd_env'])[0+1:pre_proc_data_24160_orig.shape[1]+1])
envelopes_24160Q16.append(np.squeeze(hdf5storage.loadmat('./24160_data_Q16/swt_env_24160Q16.mat')['swt_env'])[0:pre_proc_data_24160_orig.shape[1]])
envelopes_24160Q16 = np.stack(envelopes_24160Q16, axis=0)

#40840 Q32
envelopes_40840Q32 = []
envelopes_40840Q32.append(np.squeeze(hdf5storage.loadmat('./40840_data/homo_env_40840.mat')['homo_env'])[0:pre_proc_data_40840_orig.shape[1]])
envelopes_40840Q32.append(np.squeeze(hdf5storage.loadmat('./40840_data/hilb_env_40840.mat')['hilb_env'])[0+1:pre_proc_data_40840_orig.shape[1]+1])
envelopes_40840Q32.append(np.squeeze(hdf5storage.loadmat('./40840_data/psd_env_40840.mat')['psd_env'])[0+1:pre_proc_data_40840_orig.shape[1]+1])
envelopes_40840Q32.append(np.squeeze(hdf5storage.loadmat('./40840_data/swt_env_40840.mat')['swt_env'])[0:pre_proc_data_40840_orig.shape[1]])
envelopes_40840Q32 = np.stack(envelopes_40840Q32, axis=0)

#40840 Q16
envelopes_40840Q16 = []
envelopes_40840Q16.append(np.squeeze(hdf5storage.loadmat('./40840_data_Q16/homo_env_40840Q16.mat')['homo_env'])[0:pre_proc_data_40840_orig.shape[1]])
envelopes_40840Q16.append(np.squeeze(hdf5storage.loadmat('./40840_data_Q16/hilb_env_40840Q16.mat')['hilb_env'])[0+1:pre_proc_data_40840_orig.shape[1]+1])
envelopes_40840Q16.append(np.squeeze(hdf5storage.loadmat('./40840_data_Q16/psd_env_40840Q16.mat')['psd_env'])[0+1:pre_proc_data_40840_orig.shape[1]+1])
envelopes_40840Q16.append(np.squeeze(hdf5storage.loadmat('./40840_data_Q16/swt_env_40840Q16.mat')['swt_env'])[0:pre_proc_data_40840_orig.shape[1]])
envelopes_40840Q16 = np.stack(envelopes_40840Q16, axis=0)

In [4]:
def printMetrics(original_envelop, envelops):
    print('SWT Correlation:',"{:.5f}".format(stats.pearsonr(original_envelop[3], envelops[3]).statistic))
    print('HOMO Correlation: ',"{:.5f}".format(stats.pearsonr(original_envelop[0], envelops[0]).statistic))
    print('HILB Correlation: ',"{:.5f}".format(stats.pearsonr(original_envelop[1], envelops[1]).statistic))
    print('PSD Correlation: ',"{:.5f}".format(stats.pearsonr(original_envelop[2], envelops[2]).statistic))
    print()
    print('SWT MSE: ',"{:.5f}".format(mean_squared_error(original_envelop[3], envelops[3])))
    print('HOMO MSE: ',"{:.5f}".format(mean_squared_error(original_envelop[0], envelops[0])))
    print('HILB MSE: ',"{:.5f}".format(mean_squared_error(original_envelop[1], envelops[1])))
    print('PSE MSE: ',mean_squared_error(original_envelop[2], envelops[2]))

In [5]:
print('2530 Q32 RECORDING')
printMetrics(pre_proc_data_2530_orig, envelopes_2530Q32)
print('-----')
print('2530 Q16 RECORDING')
printMetrics(pre_proc_data_2530_orig, envelopes_2530Q16)
print('-----')
print('14241 Q32 RECORDING')
printMetrics(pre_proc_data_14241_orig, envelopes_14241Q32)
print('-----')
print('14241 Q16 RECORDING')
printMetrics(pre_proc_data_14241_orig, envelopes_14241Q16)
print('-----')
print('23625 Q32 RECORDING')
printMetrics(pre_proc_data_23625_orig, envelopes_23625Q32)
print('-----')
print('23625 Q16 RECORDING')
printMetrics(pre_proc_data_23625_orig, envelopes_23625Q16)
print('-----')
print('24160 Q32 RECORDING')
printMetrics(pre_proc_data_24160_orig, envelopes_24160Q32)
print('-----')
print('24160 Q16 RECORDING')
printMetrics(pre_proc_data_24160_orig, envelopes_24160Q16)
print('-----')
print('40840 Q32 RECORDING')
printMetrics(pre_proc_data_40840_orig, envelopes_40840Q32)
print('-----')
print('40840 Q16 RECORDING')
printMetrics(pre_proc_data_40840_orig, envelopes_40840Q16)
print('-----')


2530 Q32 RECORDING
SWT Correlation: 0.67362
HOMO Correlation:  0.67493
HILB Correlation:  0.77181
PSD Correlation:  0.33401

SWT MSE:  0.00076
HOMO MSE:  0.00088
HILB MSE:  0.00076
PSE MSE:  2.8686299665125093e-09
-----
2530 Q16 RECORDING
SWT Correlation: 0.67466
HOMO Correlation:  0.67743
HILB Correlation:  0.77181
PSD Correlation:  0.31862

SWT MSE:  0.00076
HOMO MSE:  0.00087
HILB MSE:  0.00076
PSE MSE:  2.7453227627765493e-09
-----
14241 Q32 RECORDING
SWT Correlation: 0.79760
HOMO Correlation:  0.63893
HILB Correlation:  0.89513
PSD Correlation:  0.86764

SWT MSE:  0.00005
HOMO MSE:  0.00035
HILB MSE:  0.00014
PSE MSE:  7.999806294464213e-11
-----
14241 Q16 RECORDING
SWT Correlation: 0.79586
HOMO Correlation:  0.62915
HILB Correlation:  0.89514
PSD Correlation:  0.86484

SWT MSE:  0.00005
HOMO MSE:  0.00037
HILB MSE:  0.00014
PSE MSE:  7.896593729953926e-11
-----
23625 Q32 RECORDING
SWT Correlation: 0.72906
HOMO Correlation:  0.72450
HILB Correlation:  0.81950
PSD Correlation:  0.6